In [223]:
import scipy.io as scio
import scipy.sparse as scsp
import numpy as np
import numdifftools as nd
from newton_method import nr_algo

In [78]:
def kd(i,j):
    if i == j:
        return 1.
    else:
        return 0.

In [79]:
def logposterior(y, C, d, A, B, q, q0, u, n_time_steps, n_neurons, n_stimuli_dims):
    """
    :param y: neuron capture data
    :param C: latent space to neuron space transformation matrix
    :param d: mean firing rates
    :param A: deterministic component of the evolution state[t] to state[t+1]
    :param q: covariance of the innovations that perturb the latent state at each time step
    :param q0: covariance of the initial state x1 of each trial
    :param B: mapping of stimuli to "latent space stimuli"
    :param u: stimuli (4-D one-hot)
    :param n_time_steps: number of time steps
    :param n_neurons: number of neurons
    :return: the log-posterior of eq.4 in Macke et al. 2011
    """

    def logpost(x):

        # first compute useful values
        q0inv = np.linalg.inv(q0)
        qinv = np.linalg.inv(q)
        
        constants = .5 * (n_time_steps-1) * np.log(np.abs(np.linalg.det(q))) + .5 * np.log(np.abs(np.linalg.det(q0)))
        term1 = sum(y[t*n_neurons:(t+1)*n_neurons].T @ (C @ x[t*nld:(t+1)*nld] + d) -
                    np.sum(np.exp(C @ x[t*nld:(t+1)*nld] + d)) for t in range(n_time_steps-1))
        term2 = - .5 * (x[1*nld:(1+1)*nld] - x[0*nld:(0+1)*nld]).T @ q0inv @ (x[1*nld:(1+1)*nld] - x[0*nld:(0+1)*nld])

        term3 = - .5 * sum((x[(t+1)*nld:(t+2)*nld] -
                    A @ x[t*nld:(t+1)*nld] -
                    B @ u[t*n_stimuli_dims:(t+1)*n_stimuli_dims]).T @ qinv @ (x[(t+1)*nld:(t+2)*nld] -
                    A @ x[t*nld:(t+1)*nld] -
                    B @ u[t*n_stimuli_dims:(t+1)*n_stimuli_dims])
                    for t in range(n_time_steps - 1))

        return constants + term1 + term2 + term3

    return logpost

In [145]:
def logposteriorDerivative(y, C, d, A, B, q, q0, u, n_time_steps, n_neurons, n_stimuli_dims, nld):
    def f(x):
        df = np.empty_like(x)
        Qinv = np.linalg.inv(q)
        Q0inv = np.linalg.inv(q0)
        for t in range(1, n_time_steps):
            df[t*nld:(t+1)*nld] = sum((y[t*n_neurons:(t+1)*n_neurons][i] - np.exp(C[i]@x[t*nld:(t+1)*nld] + d[i]))*C[i] 
             for i in range(n_neurons)) + A.T @ Qinv @ (x[t*nld:(t+1)*nld] - A @ x[t*nld:(t+1)*nld] - B @ u[t*n_stimuli_dims:(t+1)*n_stimuli_dims]) \
            - Qinv @ (x[t*nld:(t+1)*nld] - A @ x[(t-1)*nld:t*nld] - B @ u[(t-1)*n_stimuli_dims:t*n_stimuli_dims]) \
            - kd(t, 1) * (Q0inv @ (x[1*nld:(1+1)*nld] - x[0*nld:(0+1)*nld]))
        return df
    return f

In [143]:
llpd = logposteriorDerivative(y, C, d, A, B, q, q0, u, n_time_steps, n_neurons, n_stimuli_dims, nld):
    

In [144]:
llpd(mu)

[ 0.45458436 -3.52913903 -0.23777874 -0.27522849  6.07159269]


array([   0.        ,    0.        ,    0.        , ...,  547.2788291 ,
        282.81348064,  -56.99468261])

In [212]:
def logposteriorHessian(y, C, d, A, B, q, q0, u, n_time_steps, n_neurons, n_stimuli_dims, nld):
    def f(x):
        h = np.zeros((n_time_steps*nld, n_time_steps*nld))
        Qinv = np.linalg.inv(q)
        Q0inv = np.linalg.inv(q0)
        h[1*nld:(1+1)*nld, 1*nld:(1+1)*nld] += - Q0inv
        
        for t in range(1, n_time_steps-1):
            h[t*nld:(t+1)*nld, t*nld:(t+1)*nld] += -sum(np.exp(C[i]*x[t*nld:(t+1)*nld] + d[i]) * np.outer(C[i], C[i].T) 
                                          for i in range(n_neurons)) + A.T @ Qinv @ A - Qinv
            h[t*nld:(t+1)*nld, (t+1)*nld:(t+2)*nld] += - A.T @ Qinv
            h[t*nld:(t+1)*nld, (t-1)*nld:t*nld] += - Qinv @ A
            
        h[n_time_steps*nld:(n_time_steps+1)*nld, n_time_steps*nld:(n_time_steps+1)*nld] += \
        -sum(np.exp(C[i]*x[n_time_steps*nld:(n_time_steps+1)*nld] + d[i]) * np.outer(C[i], C[i].T) for i in range(n_neurons)) \
        + A.T @ Qinv @ A - Qinv
        h[n_time_steps*nld:(n_time_steps+1)*nld, (n_time_steps-1)*nld:n_time_steps*nld] += - Qinv @ A
        return h
    return f


def logposteriorhessianoptomized(y, C, d, A, B, q, q0, u, n_time_steps, n_neurons, n_stimuli_dims, nld):
    def f(x):
        Qinv = np.linalg.inv(q)
        Q0inv = np.linalg.inv(q0)
        
        diag = []
        off_diag = []
        diag += -[Q0inv]
        ATQinvA = A.T @ Qinv @ A
        ATQinv = A.T @ Qinv
        QinvA = Qinv @ A
        ATQinvAminusQinv = ATQinvA - Qinv
        diag.append(- QinvA)
        for t in range(0, n_time_steps-1):
            
            diag.append(scsp.coo_matrix(-sum(np.exp(C[i]*x[t*nld:(t+1)*nld] + d[i]) * np.outer(C[i], C[i].T) 
                                          for i in range(n_neurons)) + ATQinvAminusQinv))
            off_diag.append(scsp.coo_matrix(-ATQinv))
        
        
        h = scsp.block_diag(diag)
        od = scsp.block_diag(off_diag)
        h[n_time_steps:2 * ntimesteps ]
        
        return h
    return f
    

In [213]:
llph = logposteriorHessian(y, C, d, A, B, q, q0, u, n_time_steps, n_neurons, n_stimuli_dims, nld)

In [220]:
tl = llph(mu)

KeyboardInterrupt: 

In [82]:
def jllmake(y, n_stimuli_dims, n_neurons, n_time_steps, mu, cov, Q, Q0, x0, A, u, B):
    def jointloglikelihood(dC):

        d = np.empty(n_neurons)
        C = np.empty((n_neurons, nld))

        for i in range(n_neurons+1):
            d[i] = dC[i*(nld+1)]
            C[i] = dC[i*(nld+1) + 1:i*(nld+1)]

        # precompute for efficiency
        q0inv = np.linalg.inv(Q0)
        qinv = np.linalg.inv(Q)

        jll = sum(y[t*n_neurons:(t+1)*n_neurons].T @ C @ mu[t*nld:(t+1)*nld] \
                   + y[t*n_neurons:(t+1)*n_neurons].T @ d \
                   - .5 * np.exp(C @ mu[t*nld:(t+1)*nld] +
                                 .5 * C.T @ cov[t*n_neurons:(t+1)*n_neurons, t*n_neurons:(t+1)*n_neurons] @ C + d) \
                   - .5 * mu[nld:2*nld].T @ q0inv @ mu[nld:2*nld] \
                   + .5 * np.trace(q0inv @ cov[nld:2*nld, 1*nld:2*nld]) \
                   + .5 * mu[nld:2*nld].T @ q0inv @ x0 \
                   + .5 * x0.T @ q0inv @ mu[nld:2*nld] \
                   - .5 * x0.T @ q0inv @ mu[nld:2*nld] \
                   - .5 * x0.T @ q0inv @ x0 \
                   - .5 * mu[(t+1)*nld:(t+2)*nld] @ q0inv @ mu[(t+1)*nld:(t+2)*nld] \
                   + np.trace(qinv @ cov[(t+1)*nld, (t+1)*nld]) \
                   + .5 * mu[(t+1)*nld:(t+2)*nld].T * qinv @ A @ mu[t*nld:(t+1)*nld] \
                   + np.trace(qinv @ A @ cov[t*nld:(t+1)*nld, (t+1)*nld:(t+2)*nld]) \
                   + .5 * mu[(t+1)*nld].T @ qinv @ B @ u[t*n_stimuli_dims:(t+1)*n_stimuli_dims] \
                   + .5 * mu[t*nld:(t+1)*nld].T @ A.T @ qinv @ mu[(t+1)*nld:(t+2)*nld] \
                   + np.trace(A @ qinv @ cov[(t+1)*nld:(t+2)*nld, t*nld:(t+1)*nld]) \
                   - .5 * mu[t*nld:(t+1)*nld].T @ A.T @ qinv @ A @ B @ u[t*n_stimuli_dims:(t+1)*n_stimuli_dims] \
                   + np.trace(A.T @ qinv @ A @ cov[t*nld:(t+1)*nld, t*nld:(t+1)*nld]) \
                   - .5 * mu[t*nld:(t+1)*nld].T @ A.T @ qinv @ B @ u[t*n_stimuli_dims:(t+1)*n_stimuli_dims] \
                   + .5 * u[t*n_stimuli_dims:(t+1)*n_stimuli_dims].T @ B.T @ qinv @ mu[(t+1)*nld:(t+2)*nld] \
                   - .5 * u[t*n_stimuli_dims:(t+1)*n_stimuli_dims].T @ B.T @ qinv @ A @ mu[(t+1)*nld:(t+2)*nld] \
                   - .5 * u[t*n_stimuli_dims:(t+1)*n_stimuli_dims].T @ B.T @ qinv @ B @ u[t*n_stimuli_dims:(t+1)*n_stimuli_dims] for t in range(n_time_steps-1)) \
              - .5 * np.log(np.abs(np.det(Q0))) - .5 * (n_time_steps-1) * np.log(np.abs(np.det(Q)))

        return jll
    return jointloglikelihood

In [83]:
def laplace_approximation(f, df, Hf, mu):
    # use NR algorithm to compute minimum of log-likelihood
    mu = nr_algo(f, df, Hf, mu)

    # negative inverse of Hessian is covariance matrix
    covariance = -np.linalg.inv(H(mu))
    return mu, covariance

In [84]:
def jllDerivative(n_neurons, nld, mu, cov, n_time_steps, y):
    def f(dC):
        d = np.empty(n_neurons)
        C = np.empty((n_neurons, nld))

        for i in range(n_neurons+1):
            d[i] = dC[i*(nld+1)]
            C[i] = dC[i*(nld+1) + 1:i*(nld+1)]

        djlld = sum(
                y[t] + np.exp(C @ mu[t*nld:(t+1)*nld] + d + .5 * np.diag(C.T @ cov[t*nld:(t+1)*nld, t*nld:(t+1)*nld] @ C))
                for t in range(n_time_steps))

        djllC = np.array([
                sum(
                y[t][i] * cov[t*nld:(t+1)*nld, t*nld:(t+1)*nld] @ C[i] + mu[t] +
                np.exp(C[i] @ mu[t*nld:(t+1)*nld] + d[i] + .5 * C[i] @ cov[t*nld:(t+1)*nld, t*nld:(t+1)*nld] @ C[i]) *
                (u[t] + cov[t*nld:(t+1)*nld, t*nld:(t+1)*nld] @ C[i])
                for t in range(n_time_steps))
                for i in range(n_neurons)])

        djlldC = np.array([np.concatenate(djlld[i], djllC[i]) for i in range(n_neurons)])
        return djlldC
    return f

In [85]:
def jllHessian(n_neurons, nld, mu, cov, n_time_steps, y):
    def f(dC):

        d = np.empty(n_neurons)
        C = np.empty((n_neurons, nld))

        for i in range(n_neurons+1):
            d[i] = dC[i*(nld+1)]
            C[i] = dC[i*(nld+1) + 1:i*(nld+1)]

        Hjll = np.zeros((n_neurons + n_neurons * nld, n_neurons + n_neurons * nld))

        for i in range(n_neurons):
            Hjll[i * (nld + 1), i * (nld + 1)] = sum(np.exp(C[i] @ mu[t*nld:(t+1)*nld] + d[i] +
                                                    .5 * C[i] @ cov[t*nld:(t+1)*nld, t*nld:(t+1)*nld] @ C[i])
                                                    for t in range(n_time_steps))

            Hjll[i * (nld + 1), i * (nld + 1) + 1:(i + 2) * (nld + 1)] = \
                sum((mu[t] + cov[t, t] @ C[i]) * np.exp(C[i] @ mu[t*nld:(t+1)*nld] + d[i] +
                    .5 * C[i] @ cov[t*nld:(t+1)*nld, t*nld:(t+1)*nld] @ C[i]) for t in range(n_time_steps))

            Hjll[i * (nld + 1) + 1:(i + 2) * (nld + 1), i * (nld + 1)] = \
                Hjll[i * (nld + 1), i * (nld + 1) + 1:(i + 2) * (nld + 1)].T

            Hjll[i * (nld + 1) + 1:(i + 2) * (nld + 1), i * (nld + 1) + 1:(i + 2) * (nld + 1)] = np.array([
                sum(y[t][i] * cov[t, t] @ C[i] + mu[t] +
                np.outer(mu[t] + cov[t, t] @ C[i], mu[t] + cov[t, t] @ C[i])[i] +
                np.exp(C[i] @ mu[t*nld:(t+1)*nld] + d[i] + .5 * C[i] @ cov[t*nld:(t+1)*nld, t*nld:(t+1)*nld] @ C[i]) *
                (u[t] + cov[t*nld:(t+1)*nld, t*nld:(t+1)*nld] @ C[i])
                for t in range(n_time_steps))
                ])
        return Hjll
    return f


In [180]:
# load data
n_time_steps = 26187
n_neurons = 300  # number of neurons
nld = 5  # number of latent dimensions
n_stimuli_dims = 4
frameHz = 10  # frames per seconds
data = scio.loadmat('data/compiled_dF033016.mat')
y = data['behavdF'].flatten()
onset = data['onsetFrame'].T[0]
resptime = data['resptime'].T[0]
correct = data['correct'][0]
orient = np.array(data['orient'][0], np.int)
location = (data['location'][0]+1)//2

# create empty u
u = np.zeros((n_time_steps, n_stimuli_dims))
# set stimuli
for ot, rt, cor, ori, loc in zip(onset, resptime, correct, orient, location):
    # compute what u should be here
    u[ot:ot+int((rt+2.75+(4.85-2.75)*(1-cor))*frameHz)] = \
        np.array([ori*loc, (1-ori)*loc, ori*(1-loc), (1-ori)*(1-loc)], np.int)

u = u.flatten()
# Initialize parameters to random values
C = np.random.randn(n_neurons* nld).reshape(-1, nld)
d = np.random.randn(n_neurons)
x0 = np.random.randn(nld)
A = np.random.randn(nld * nld).reshape(-1, nld)
q0 = np.abs(np.random.randn(nld * nld).reshape(-1, nld))
q0 = q0@q0.T
q = np.abs(np.random.randn(nld * nld).reshape(-1, nld))
q = q@q.T
B = np.random.randn(nld * n_stimuli_dims).reshape(-1, n_stimuli_dims)
mu = np.random.randn(nld*n_time_steps)
cov = np.random.randn(nld * nld).reshape(-1, nld)


/Users/BisharaKorkor/.virtualenvs/tf3/lib/python3.5/site-packages/ipykernel/__main__.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [219]:
lp = logposterior(y, C, d, A, B, q, q0, u, n_time_steps, n_neurons, n_stimuli_dims)
lpd = logposteriorDerivative(y, C, d, A, B, q, q0, u, n_time_steps, n_neurons, n_stimuli_dims, nld)
lph = logposteriorHessian(y, C, d, A, B, q, q0, u, n_time_steps, n_neurons, n_stimuli_dims, nld)

laplace_approximation(lp, lpd, lph, mu)

ValueError: operands could not be broadcast together with shapes (5,) (0,) 

In [ ]:
scsp.